## to run it 


In [11]:
# https://github.com/danielroich/PTI

# run below commands from /PTI/ 
# export PYTHONPATH=$PYTHONPATH:$PWD

# change path inside it.
# python utils/align_data.py

# path_ = "/home/bean/DragVideo/PTI"
# import os 
# os.chdir(path_)


In [2]:
# import os 
# dir = "/workspace/PTI/data/raw_images"
# # it has images from 0 to 400 .jpeg 
# # rename them so that all are 3 digits
# for i in range(400):
#     if i < 10:
#         os.rename(dir + "/" + str(i) + ".jpg", dir + "/00" + str(i) + ".jpg")
#     elif i < 100:
#         os.rename(dir + "/" + str(i) + ".jpg", dir + "/0" + str(i) + ".jpg")
#     else:
#         continue
#         # os.rename(dir + "/" + str(i) + ".jpeg", dir + "/" + str(i) + ".jpeg")


In [1]:
import os 
import sys
root_dir = '/home/bean/DragVideo'
os.chdir(f'{root_dir}/PTI')

use_multi_id_training = True

from scripts.run_pti import run_PTI
model_id = run_PTI(use_wandb=False, use_multi_id_training=use_multi_id_training)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
current_directory:  /home/bean/DragVideo/PTI


KeyboardInterrupt: 

In [2]:
import os
import sys
import pickle
import numpy as np
from PIL import Image
import torch
from configs import paths_config, hyperparameters, global_config
from utils.align_data import pre_process_images
from scripts.run_pti import run_PTI
from IPython.display import display
import matplotlib.pyplot as plt
from scripts.latent_editor_wrapper import LatentEditorWrapper

In [2]:
def load_generators(model_id, image_name):
  with open(paths_config.stylegan2_ada_ffhq, 'rb') as f:
    old_G = pickle.load(f)['G_ema'].cuda()

  with open(f'{paths_config.checkpoints_dir}/model_{model_id}_{image_name}.pt', 'rb') as f_new:
    new_G = torch.load(f_new).cuda()

  return old_G, new_G

In [9]:
# image_name = "000"

# # /home/bean/DragVideo/DragGAN/PTI_results/checkpoints/model_ZEWLQSQSSQWA_000.pt
# model_id = "ZEWLQSQSSQWA"

In [4]:
model_id = '/home/bean/DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhq-1024_module.pt'
use_multi_id_training = True

generator_type = paths_config.multi_id_model_type if use_multi_id_training else image_name
old_G, new_G = load_generators(model_id, generator_type)

FileNotFoundError: [Errno 2] No such file or directory: '/home/bean/DragVideo//DragGAN/PTI_results/checkpoints/model_HCQDSNSTNNJM_multi_id.pt'

In [5]:
#code from : https://github.com/danielroich/PTI/issues/26 , plus little bit modification

def export_updated_pickle(new_G,model_id):
  print("Exporting large updated pickle based off new generator and ffhq.pkl")
  with open(paths_config.stylegan2_ada_ffhq, 'rb') as f:
    d = pickle.load(f)
    old_G = d['G_ema'].cuda()
    old_D = d['D'].eval().requires_grad_(False).cpu()

  tmp = {}
  tmp['G'] = old_G.eval().requires_grad_(False).cpu()
  tmp['G_ema'] = new_G.eval().requires_grad_(False).cpu()
  tmp['D'] = old_D
  tmp['training_set_kwargs'] = None
  tmp['augment_pipe'] = None

  # with open(f'{paths_config.checkpoints_dir}/stylegan3_{model_id}.pkl', 'wb') as f:
  #     pickle.dump(tmp, f)

export_updated_pickle(new_G,model_id)

NameError: name 'new_G' is not defined

In [12]:
model_id # 'AXOCWYGCAEDY'for 10 images

'ZEWLQSQSSQWA'

In [63]:
# # .pt to .pkl file conversion

# sg3_pt_file_path = '/home/bean/DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhqu-1024.pt' #downloaded
# # sg3_pt_file_path_full="/home/bean/DragVideo/Data_store/data/PTI_results/checkpoints/model_HCQDSNSTNNJM_multi_id.pt"

# with open(sg3_pt_file_path, 'rb') as f_new:
#     weights = torch.load(f_new)#.cuda()

# # with open(sg3_pt_file_path_full, 'rb') as f_new:
# #     module = torch.load(f_new)#.cuda()

In [64]:
# sg3_1024_pkl = "/home/bean/DragVideo/PTI/pretrained_models/stylegan3-r-ffhq-1024x1024.pkl"
# with open(sg3_1024_pkl,'rb') as f:
#     sg3_1024 = pickle.load(f)

# torch.save(sg3_1024["G"],'/home/bean/DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhqu-1024_module.pt')

In [6]:
def export_updated_pickle(new_G,model_id):
  print("Exporting large updated pickle based off new generator and ffhq.pkl")
  with open(paths_config.stylegan2_ada_ffhq, 'rb') as f:
    d = pickle.load(f)
    old_G = d['G_ema'].cuda()
    old_D = d['D'].eval().requires_grad_(False).cpu()

  tmp = {}
  tmp['G'] = old_G.eval().requires_grad_(False).cpu()
  tmp['G_ema'] = new_G.eval().requires_grad_(False).cpu()
  tmp['D'] = old_D
  tmp['training_set_kwargs'] = None
  tmp['augment_pipe'] = None

  with open(f'{paths_config.checkpoints_dir}/stylegan2_{model_id}.pkl', 'wb') as f:
      pickle.dump(tmp, f)

model_id = '3rdtime_test' 
export_updated_pickle(new_G,model_id)

Exporting large updated pickle based off new generator and ffhq.pkl


AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [68]:
sg3_3rdtime_pt = '/home/bean/DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhq-1024_module.pt'
use_multi_id_training = True
sg3_1024_pkl = "/home/bean/DragVideo/PTI/pretrained_models/stylegan3-r-ffhq-1024x1024.pkl"



In [69]:
with open(sg3_1024_pkl, 'rb') as f:
    old_G = pickle.load(f)['G_ema'].cuda()

with open(sg3_3rdtime_pt, 'rb') as f_new:
    new_G = torch.load(f_new).cuda()

In [70]:
with open(sg3_1024_pkl, 'rb') as f:
    d = pickle.load(f)
    old_G = d['G_ema'].cuda()
    old_D = d['D'].eval().requires_grad_(False).cpu()

tmp = {}
tmp['G'] = old_G.eval().requires_grad_(False).cpu()
tmp['G_ema'] = new_G.eval().requires_grad_(False).cpu()
tmp['D'] = old_D
tmp['training_set_kwargs'] = None
tmp['augment_pipe'] = None

with open('/home/bean/DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhq-1024_module.pkl', 'wb') as f:
    pickle.dump(tmp, f)

In [4]:
sg3u_3rdtime_pt = '/home/bean/DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhqu-1024_module.pt'
use_multi_id_training = True
sg3_1024_pkl = "/home/bean/DragVideo/PTI/pretrained_models/stylegan3-r-ffhq-1024x1024.pkl"

In [5]:
with open(sg3_1024_pkl, 'rb') as f:
    old_G = pickle.load(f)['G_ema'].cuda()

with open(sg3u_3rdtime_pt, 'rb') as f_new:
    new_G = torch.load(f_new).cuda()

In [6]:
with open(sg3_1024_pkl, 'rb') as f:
    d = pickle.load(f)
    old_G = d['G_ema'].cuda()
    old_D = d['D'].eval().requires_grad_(False).cpu()

tmp = {}
tmp['G'] = old_G.eval().requires_grad_(False).cpu()
tmp['G_ema'] = new_G.eval().requires_grad_(False).cpu()
tmp['D'] = old_D
tmp['training_set_kwargs'] = None
tmp['augment_pipe'] = None

with open('/home/bean/DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhqu-1024_module.pkl', 'wb') as f:
    pickle.dump(tmp, f)